In [2]:
from hazm import *
import re
import fitz  
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama.chat_models import ChatOllama 
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough 

import warnings
warnings.filterwarnings('ignore')

In [3]:
class RAGFA():
    
    def __init__(self, model_name, pdf_path):
        self.pdf_path = pdf_path
        self.all_text = self.preparePDF()
        self.vector_db = self.prepareVectorDB()
        self.llm = ChatOllama(model=model_name)
        self.QUERY_PROMPT = PromptTemplate(
            input_variables=["question"],
            template="""به عنوان یک دستیار هوش مصنوعی، نقش شما این است که پنج نسخه جایگزین از 
            سوال داده شده را تولید کنید. این تغییرات باید عبارت‌ها و یا نحوه بیان مختلفی داشته باشند تا دیدگاه‌های 
            متنوعی را پوشش دهند و بازیابی اطلاعات مرتبط از پایگاه داده وکتور را آسان‌تر کنند، به ویژه در مواقعی که جستجوی 
            مشابهت ممکن است محدودیت‌هایی داشته باشد.
            
            لطفاً این سوالات جایگزین را هرکدام در یک خط جدید بنویسید.
            
            سوال اصلی: {question}""",
        )
        self.retriever = MultiQueryRetriever.from_llm(self.vector_db.as_retriever(), self.llm, prompt=self.QUERY_PROMPT)
        self.template = """شما یک استاد دانشگاه با تخصص در تاریخ ایران و ادبیات ایران و بخصوص شاهنامه هستید و مدرکی در این زمینه دارید. 
                    شما سوال زیر را برای دانشجویانتان تعیین کرده‌اید و حالا دارید یک پاسخ واضح و آسان‌فهم برای آن ارائه می‌دهید.
                    لطفاً پاسخ سوال را تنها بر اساس متن زیر از کتاب *شاهنامه* بنویسید.
                    
                    لطفاً سعی کنید پاسخ را مانند یک حرفه‌ای ارائه دهید و در صورت امکان از مثال‌ها استفاده کنید.
                    به هیچ عنوان از کلمات انگلیسی استفاده نکن
                    زمینه: {context}
                    
                    سوال: {question}
                    """
        self.prompt = ChatPromptTemplate.from_template(self.template)
        self.chain = (
            {"context": self.retriever, "question": RunnablePassthrough()}
            | self.prompt
            | self.llm
            | StrOutputParser()
        )

        
    def prepareVectorDB(self):
        embeddings = HuggingFaceEmbeddings(model_name='HooshvareLab/bert-fa-zwnj-base')
        chunks = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100).split_text(self.all_text)
        documents = [Document(page_content=chunk) for chunk in chunks]
        vector_db = Chroma.from_documents(documents, embeddings)
        return vector_db
        
    def preparePDF(self):
        doc = fitz.open(self.pdf_path)
        all_text = ""
        for page in doc:
            text = page.get_text()
            text = text.replace('\n', ' ')
            all_text += self.preprocess(text)
            
        return all_text
    
    def invoke(self, query):
        return self.chain.invoke(query)
        
    def preprocess(self, text, stopwords=None, normalizer=False, lemmatizer=False, stemmer=False):
        normalizer = Normalizer(correct_spacing=True, remove_diacritics=True, remove_specials_chars=True, unicodes_replacement=True)
        lemmatizer = Lemmatizer()
        stemmer = Stemmer()

        def remove_stopwords(text, stopwords):
            text=str(text)
            filtered_tokens = [token for token in text.split() if token not in stopwords]
            filtered_text = ' '.join(filtered_tokens)
            return filtered_text

        def remove_emoji(text): 
            emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        u"\U0001f926-\U0001f937"
                        u'\U00010000-\U0010ffff'
                        u"\u200d"
                        u"\u200c"
                        u"\u2640-\u2642"
                        u"\u2600-\u2B55"
                        u"\u23cf"
                        u"\u23e9"
                        u"\u231a"
                        u"\u3030"
                        u"\ufe0f"
            "]+", flags=re.UNICODE)
            
            return emoji_pattern.sub(r' ', text)

        def remove_halfspace(text): 
            emoji_pattern = re.compile("["                
                        u"\u200c"              
            "]+", flags=re.UNICODE)
            
            return emoji_pattern.sub(r' ', text) 

        def remove_link(text): 
            return re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', str(text))

        def remove_picUrl(text):
            return re.sub(r'pic.twitter.com/[\w]*',"", str(text))

        def remove_rt(text):
            z = lambda text: re.compile('\#').sub('', re.compile('RT @').sub('@', str(text), count=1).strip())
            return z(text)

        def remove_hashtag(text):
            return re.sub(r"#[^\s]+", '', str(text))

        def remove_mention(text):
            return re.sub(r"@[^\s]+", '', str(text))

        def remove_email(text): 
            return re.sub(r'\S+@\S+', '', str(text))

        def remove_numbers(text): 
            return re.sub(r'^\d+\s|\s\d+\s|\s\d+$', ' ', str(text))

        def remove_html(text):
            html_pattern = re.compile('<.*?>')
            return html_pattern.sub(r'', str(text))

        def remove_quote(text): 
            return  str(text).replace("'","")

        def remove_chars(text): 
            return  re.sub(r'[$+&+;+]|[><!+،:’,\(\).+]|[-+]|[…]|[\[\]»«//]|[\\]|[#+]|[_+]|[—+]|[*+]|[؟+]|[?+]|[""]', ' ', str(text))

        def remove_englishword(text): 
            return re.sub(r'[A-Za-z]+', '', str(text))

        def remove_extraspaces(text):
            return re.sub(r' +', ' ', text)

        def remove_extranewlines(text):
            return re.sub(r'\n\n+', '\n\n', text)

        def lemmatizer_text(text):
            words = []
            for word in text.split():
                words.append(lemmatizer.lemmatize(word))
            return ' '.join(words)

        def stemmer_text(text):
            words = []
            for word in text.split():
                words.append(stemmer.stem(word))
            return ' '.join(words)

        def normalizer_text(text, normalize, lemmatize, stemm):
            if normalize:
                text = normalizer.normalize(text)
            if stemm:
                text = stemmer_text(text)
            if lemmatize:
                text = lemmatizer_text(text)
            return text

        def preprocess_text(text, stopwords=None, normalizer=False, lemmatizer=False, stemmer=False):
            text = remove_link(text)
            text = remove_picUrl(text)
            text = remove_englishword(text)
            if stopwords:
                text = remove_stopwords(text, stopwords)
            text = remove_emoji(text)
            text = remove_rt(text)
            text = remove_mention(text)
            text = remove_emoji(text)
            text = remove_hashtag(text)   
            text = remove_email(text) 
            text = remove_html(text) 
            text = remove_chars(text)
            text = remove_numbers(text)
            text = remove_quote(text)
            text = remove_extraspaces(text)
            text = remove_extranewlines(text)
            text = remove_halfspace(text) 
            text = normalizer_text(text, normalizer, lemmatizer, stemmer)
            return text
        
        return preprocess_text(text, stopwords=None, normalizer=False, lemmatizer=False, stemmer=False)

In [4]:
pdf_path='arash.pdf'
# model_name = "llama3.2:1b"
model_name = 'partai/dorna-llama3:8b-instruct-q4_0'

rag = RAGFA(model_name, pdf_path)

No sentence-transformers model found with name HooshvareLab/bert-fa-zwnj-base. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
query = ''
# query = 'دلیل اینکه داستان آرش در شاهنامه نیست چیه؟'
# query = 'چرا آرش در شاهنامه فردوسی نیست؟'
# query = 'آرش کی بود؟'
query = 'متن رو خلاصه کن'


text = rag.invoke(query)
print(text)

در جواب، متنی که خلاصه شده است. 

در ابتدا، داستان آرش کمانگیر شجاع و برترین فرماندهان نظامی ایرانه در تاریخ شناخته شده است. او تیر خود را برای برجسته کردن افتخار خود از مرزهای دور دست برای هدف برد و طوالنی ترین راهی را که هیچکس قبل از او طی نکرده بود، انتخاب کرد و به عنوان یکی از اسطوره های باشکوه تاریخ شناخته شد.

سپس داستان آرش کمانگیر با دیدن زیبایی و خوبی زنان تورانی تصمیم گرفت که با آن ها مراقبت کند و از آن ها کمک کند تا سازگار شوند و نشانگر انسانیت و مدارا در برخورد با بردگان بود. 

در پايان، داستانی که شامل ماجراجویی پراز افتخار و شجاعت است و در برابر دشمن او را به یک قهرمان تبدیل کرد.


In [8]:
queries = [
    'آرش که بود؟', 
    'کمان آرش از چی ساخته شده بود؟'
]

with open('output.txt', 'a', encoding='utf-8') as f:
    for query in queries:
            output = rag.invoke(query) 
            f.write(f'\nجواب:\n {output}')
